In [1]:
import os
 
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
 
import keras
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
from keras import optimizers

import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from keras import regularizers



(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

Using plaidml.keras.backend backend.
(50000, 32, 32, 3)


In [2]:
classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32,(3,3),input_shape=(32,32,3),padding='same',activation='sigmoid'))
classifier.add(BatchNormalization())##BatchNormalization
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(32,(3,3),activation='sigmoid',padding='same'))
classifier.add(BatchNormalization())##BatchNormalization
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(output_dim=100,activation='sigmoid',kernel_regularizer=regularizers.l2(l=0.001))) ##regularizers
classifier.add(BatchNormalization()) ##BatchNormalization
classifier.add(Dropout(p=0.5)) ##Dropout

classifier.add(Dense(output_dim=100,activation='sigmoid',kernel_regularizer=regularizers.l2(0.001)))



##regularizers
classifier.add(BatchNormalization()) ##BatchNormalization

classifier.add(Dropout(p=0.3))##Dropout

classifier.add(Dense(output_dim=10,activation='softmax'))


#超過兩個就要選categorical_crossentrophy
adam = optimizers.Adam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=1e-6, amsgrad=False)
classifier.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])


from keras.preprocessing.image import ImageDataGenerator ##Augmentation
img_gen = ImageDataGenerator( featurewise_center=True,featurewise_std_normalization=True,rotation_range=10,width_shift_range=0.1,
                                            height_shift_range=0.1,shear_range=0.1,zoom_range=0.1,horizontal_flip=True,vertical_flip=False,dtype=np.float32)
img_gen.fit(x_train)


from keras.callbacks import EarlyStopping
earlystop = EarlyStopping(monitor='test_loss', patience=8, verbose=1) ##earlystop

##開始訓練
classifier.fit_generator(img_gen.flow(x_train, y_train, batch_size=100) ,steps_per_epoch=500,
                               epochs=100, validation_data = (x_test, y_test),callbacks = [earlystop])

INFO:plaidml:Opening device "metal_amd_radeon_pro_5300m.0"
<ipython-input-2-6c9597afee79>:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", kernel_regularizer=<keras.reg..., units=100)`
  classifier.add(Dense(output_dim=100,activation='sigmoid',kernel_regularizer=regularizers.l2(l=0.001))) ##regularizers
<ipython-input-2-6c9597afee79>:19: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.5)`
  classifier.add(Dropout(p=0.5)) ##Dropout
<ipython-input-2-6c9597afee79>:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", kernel_regularizer=<keras.reg..., units=100)`
  classifier.add(Dense(output_dim=100,activation='sigmoid',kernel_regularizer=regularizers.l2(0.001)))
<ipython-input-2-6c9597afee79>:28: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.3)`
  classifier.add(Dropout(p=0.3))##Dropout
<ipython-input-2-6c9597afee79>:30: UserWarning: Update your `Dense`

TypeError: object of type 'NoneType' has no len()

## 上方有用到一些避免Overfitting 的方法，有興趣的學員們可以參考這篇Medium:https://medium.com/@CinnamonAITaiwan/cnn%E5%85%A5%E9%96%80-overfitting-d10acd15ec21

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [7]:
input_example=(np.zeros(shape=(1,32,32,3))-x_train)/(x_train+1e-7) 


array([[0.01734307, 0.02266115, 0.19550554, ..., 0.09270356, 0.00697742,
        0.02662526],
       [0.01734309, 0.02266116, 0.1955054 , ..., 0.09270365, 0.00697743,
        0.02662526],
       [0.0173431 , 0.02266116, 0.19550565, ..., 0.09270362, 0.00697743,
        0.02662524],
       ...,
       [0.01734311, 0.02266116, 0.19550565, ..., 0.09270364, 0.00697743,
        0.02662525],
       [0.01734311, 0.02266117, 0.19550557, ..., 0.09270361, 0.00697743,
        0.02662526],
       [0.0173431 , 0.02266118, 0.1955057 , ..., 0.09270364, 0.00697742,
        0.02662526]], dtype=float32)

In [8]:
classifier.predict(input_example)

array([[0.01734307, 0.02266115, 0.19550554, ..., 0.09270356, 0.00697742,
        0.02662526],
       [0.01734309, 0.02266116, 0.1955054 , ..., 0.09270365, 0.00697743,
        0.02662526],
       [0.0173431 , 0.02266116, 0.19550565, ..., 0.09270362, 0.00697743,
        0.02662524],
       ...,
       [0.01734311, 0.02266116, 0.19550565, ..., 0.09270364, 0.00697743,
        0.02662525],
       [0.01734311, 0.02266117, 0.19550557, ..., 0.09270361, 0.00697743,
        0.02662526],
       [0.0173431 , 0.02266118, 0.1955057 , ..., 0.09270364, 0.00697742,
        0.02662526]], dtype=float32)